In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
NGIDS = pd.read_csv("./NGIDS_host_log.csv")

In [3]:
def vc_mapping(DF, column, start=1):
    df_tmp = DF
    tmp = DF[column].value_counts().index
    print(column, len(tmp))
    dict_tmp = dict()
    for i, t in enumerate(tmp):
        dict_tmp[t] = i+start
    DF[column] = DF[column].map(dict_tmp)
    return DF, len(tmp)

In [4]:
NGIDS, date_len = vc_mapping(NGIDS, "date_time")
NGIDS, path_len = vc_mapping(NGIDS, "path")
NGIDS, sys_len = vc_mapping(NGIDS, "sys_call")
NGIDS, pro_len = vc_mapping(NGIDS, "pro_id")
# NGIDS = vc_mapping(NGIDS, "attack_subcat")

date_time 367290
path 100
sys_call 122
pro_id 5576


In [5]:
DT_VC = NGIDS['date_time'].value_counts()

In [6]:
DT_VC

1         2257
2         2238
3         1810
4         1792
5         1711
          ... 
367121       1
365407       1
365404       1
367122       1
366738       1
Name: date_time, Length: 367290, dtype: int64

In [7]:
tmp = [72, 113, 135, 162, 198, 371, 387]

In [8]:
for t in tmp:
    print(t," : ",sum(DT_VC.values >= t))

72  :  360152
113  :  358571
135  :  298254
162  :  223483
198  :  199948
371  :  80380
387  :  46154


In [9]:
NGIDS_1 = NGIDS[NGIDS['label']==1]

In [10]:
DT_VC_1 = NGIDS_1['date_time'].value_counts()

In [11]:
DT_VC_1

84        1017
89        1012
90        1007
101        980
123        952
          ... 
366709       1
366089       1
365154       1
365803       1
365456       1
Name: date_time, Length: 5081, dtype: int64

In [12]:
for t in tmp:
    print(t," : ",sum(DT_VC_1.values >= t))

72  :  4964
113  :  4942
135  :  4073
162  :  3064
198  :  2721
371  :  1110
387  :  706


In [13]:
NGIDS

,Unnamed: 0,date_time,pro_id,path,sys_call,event_id,attack_cat,attack_subcat,label
0,0,361840,23,15,6,45350,0,0,0
1,1,361840,32,15,6,45351,0,0,0
2,2,361840,18,6,6,45353,0,0,0
3,3,361840,20,16,9,45354,0,0,0
4,4,361840,17,6,6,45355,0,0,0
...,...,...,...,...,...,...,...,...,...
90054234,90054234,360199,2825,23,17,102975641,0,0,0
90054235,90054235,360199,2825,23,3,102975643,0,0,0
90054236,90054236,360199,53,26,15,102975644,0,0,0
90054237,90054237,360199,3,2,4,102975646,0,0,0


In [14]:
from collections import Counter
from IPython.display import display, clear_output

In [15]:
# NGIDS = NGIDS[40000367:]

In [15]:
DATE_TIME = NGIDS["date_time"].values.astype("int32")
ATTACK_CAT = NGIDS["attack_cat"].values.astype("int16")
# ATTACK_SUBCAT = NGIDS["attack_subcat"].values.astype("int16")
# PATH = NGIDS["path"].values.astype("int16")
# SYS_CALL = NGIDS["sys_call"].values.astype("int16")

In [16]:
PS_V = NGIDS[["path", "sys_call"]].values
PS_DD_V = NGIDS[["path", "sys_call"]].drop_duplicates().values

PS_DICT = dict()
for i, V in enumerate(PS_DD_V):
    PS_DICT[tuple(V)] = i+1
print(len(PS_DICT))

PS = np.zeros(len(PS_V))
for i, V in enumerate(PS_V):
    PS[i] = PS_DICT[tuple(V)]

del PS_V, PS_DD_V, PS_DICT

1670


In [17]:
PS = PS.astype("int16")

In [18]:
del NGIDS

In [19]:
def most_com(list):
    return Counter(list).most_common(n=1)[0][0]

In [20]:
def sliding_window_arr(vecs, Arr, window=5, jump=10):
    Arr_len = len(Arr)
    y_len = int(((Arr_len-window)/jump) + 1)
    res = vecs
    for i in range(y_len):
        res.append(Arr[i*jump:i*jump + window])
    return res, y_len

In [21]:
def padding(re, len_tmp, cut):
    re = np.array(re)
    if len_tmp>=cut:
        re = re[:cut]
    else:
        tmp = np.zeros(cut)
        tmp[:len(re)] = re
        re = tmp
    return re

In [22]:
def make_SW_vector(WINDOW, JUMP_1, JUMP_0):
    vecs_PS = []
    atcas = []
    start = 0
    end = 0
    check = DATE_TIME[0]
    for i, DT in enumerate(DATE_TIME):
        if (check != DT) | (i == (len(DATE_TIME) - 1)):
            check = DT
            end = i
            len_tmp = end-start

            l_val = most_com(ATTACK_CAT[start:end])
            if l_val!=(np.mean(ATTACK_CAT[start:end])):
                print("ATTACK_CAT_ERR!!", DT)
                break

            if len_tmp < WINDOW:
#                 vecs_sys_call.append(padding(SYS_CALL[start:end], len_tmp, WINDOW))
#                 vecs_path.append(padding(PATH[start:end], len_tmp, WINDOW))
#                 atcas.append(l_val)
                start = i
                continue

            if l_val != 0:
                vecs_PS, y_len = sliding_window_arr(vecs_PS, PS[start:end], WINDOW, JUMP_1)
            else:
                vecs_PS, y_len = sliding_window_arr(vecs_PS, PS[start:end], WINDOW, JUMP_0)

            for v in range(y_len):
                atcas.append(l_val)

            start = i
        if i%1000000 == 0:
            print("processing...",i, JUMP_1)
            clear_output(wait=True)
    
    col = []
    for i in range(WINDOW):
        col.append("sys"+str(i+1))
    vecs_PS = pd.DataFrame(vecs_PS, columns=col)

    vecs_PS["attack_cat"] = atcas
    vecs_PS = vecs_PS.drop_duplicates()
    return vecs_PS

In [24]:
for t in [135, 162]:
    file_name = "./NGIDS_PAT_sliding_"+str(int(t))+"_"+str(8)+"_2.csv"
    make_SW_vector(t, 8, 8).to_csv(file_name, index=False)

processing... 50000000 8
